# Prétraitement des Images Radiographiques Pulmonaires

Ce notebook réalise le prétraitement des images de radiographie pulmonaire pour la détection de pneumonie:

1. Nettoyage des images
2. Normalisation et transformation
3. Redimensionnement à une taille uniforme
4. Conversion des images RGB en niveaux de gris
5. Préparation des ensembles de données

In [ ]:
# Bibliothèques standard
import os                  
# Gestion des chemins de fichiers et des opérations sur les répertoires

import numpy as np         
# Traitement de tableaux numériques et opérations mathématiques

import pandas as pd        
# Analyse et manipulation de données tabulaires

from collections import Counter  
# Comptage d'éléments et statistiques

import time                
# Mesure de temps d'exécution

import shutil              
# Opérations avancées sur les fichiers (copie, déplacement)

import random              
# Génération de nombres aléatoires et échantillonnage

from PIL import Image      
# Chargement, manipulation et enregistrement d'images

import matplotlib.pyplot as plt  
# Création de graphiques et visualisation de données

%matplotlib inline       

## Définition des chemins et paramètres
Définition des chemins source et destination pour les images originales et prétraitées, ainsi que la taille cible (256×256) pour le redimensionnement uniforme des radiographies.

In [ ]:

base_path = 'chest_Xray'
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')
test_dir = os.path.join(base_path, 'test')

preprocessed_base = 'preprocessed_chest_xray'
preprocessed_train = os.path.join(preprocessed_base, 'train')
preprocessed_val = os.path.join(preprocessed_base, 'val')
preprocessed_test = os.path.join(preprocessed_base, 'test')

TARGET_SIZE = (256, 256) 

Création d'une structure parallèle de dossiers (`preprocessed_chest_xray`) qui accueillera les versions prétraitées des images tout en conservant la même organisation que les données originales.RetryClaude can make mistakes. Please double-check responses.

In [ ]:
def create_dirs():
    """Crée les dossiers pour stocker les images prétraitées"""
    os.makedirs(preprocessed_base, exist_ok=True)
    os.makedirs(preprocessed_train, exist_ok=True)
    os.makedirs(preprocessed_val, exist_ok=True)
    os.makedirs(preprocessed_test, exist_ok=True)
    
    os.makedirs(os.path.join(preprocessed_train, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_train, 'PNEUMONIA'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_val, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_val, 'PNEUMONIA'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_test, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_test, 'PNEUMONIA'), exist_ok=True)
    
    print("Dossiers de destination créés avec succès.")
create_dirs()

# Redistribution et prétraitement des images

Redistribution des images pour obtenir un ratio 80/10/10 (train/validation/test) tout en appliquant le prétraitement à chaque image. Cette approche corrige le déséquilibre initial (89/0.3/10.7) pour permettre une validation plus robuste du modèle.

In [ ]:
def preprocess_image(img_path, output_path, target_size=TARGET_SIZE):
    """
    Prétraite une image radiographique pour l'apprentissage automatique
    """
    try:

        img = Image.open(img_path)
        
        if img.mode == 'RGB':
            img = img.convert('L')
        
        img = img.resize(target_size, Image.LANCZOS)
        

        img_array = np.array(img)
        

        img_array = img_array / 255.0
        
        normalized_img = Image.fromarray((img_array * 255).astype(np.uint8))
        

        normalized_img.save(output_path)
    
        return True
    except Exception as e:
        print(f"Erreur lors du prétraitement de {img_path}: {e}")
        return False

# Application du prétraitement à toutes les images

Parcours de l'ensemble du dataset (train, val, test) pour appliquer la fonction de prétraitement à chaque image et sauvegarder les versions transformées dans les dossiers correspondants. Ce processus préserve l'organisation des données tout en standardisant les caractéristiques des images.

In [ ]:
def create_optimized_balanced_dataset():
    """
    Crée un dataset parfaitement équilibré 50/50 tout en conservant TOUTES les images originales
    """
    
    if os.path.exists(preprocessed_base):
        print(f"Suppression du dossier {preprocessed_base} existant...")
        shutil.rmtree(preprocessed_base)
        print("Dossier supprimé avec succès.")
    
    
    os.makedirs(preprocessed_base, exist_ok=True)
    os.makedirs(preprocessed_train, exist_ok=True)
    os.makedirs(preprocessed_val, exist_ok=True)
    os.makedirs(preprocessed_test, exist_ok=True)
    os.makedirs(os.path.join(preprocessed_train, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_train, 'PNEUMONIA'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_val, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_val, 'PNEUMONIA'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_test, 'NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_test, 'PNEUMONIA'), exist_ok=True)
    print("Dossiers recréés avec succès.")
    
    
    total_processed = 0
    successful = 0
    failed = 0
    
    start_time = time.time()
    print("Création d'un dataset optimisé avec équilibre 50/50...")
    
    
    normal_files = []
    pneumonia_files = []
    
    for folder in ['train', 'val', 'test']:
        
        normal_path = os.path.join(base_path, folder, 'NORMAL')
        if os.path.exists(normal_path):
            for f in os.listdir(normal_path):
                if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(normal_path, f)
                    if os.path.exists(img_path):
                        normal_files.append(img_path)
        
        
        pneumonia_path = os.path.join(base_path, folder, 'PNEUMONIA')
        if os.path.exists(pneumonia_path):
            for f in os.listdir(pneumonia_path):
                if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(pneumonia_path, f)
                    if os.path.exists(img_path):
                        pneumonia_files.append(img_path)
    
    
    random.shuffle(normal_files)
    random.shuffle(pneumonia_files)
    
    print(f"Total images NORMAL trouvées: {len(normal_files)}")
    print(f"Total images PNEUMONIA trouvées: {len(pneumonia_files)}")
    
    
    
    pneumonia_train_size = int(len(pneumonia_files) * 0.8)
    pneumonia_val_size = int(len(pneumonia_files) * 0.1)
    pneumonia_test_size = len(pneumonia_files) - pneumonia_train_size - pneumonia_val_size
    
    pneumonia_train = pneumonia_files[:pneumonia_train_size]
    pneumonia_val = pneumonia_files[pneumonia_train_size:pneumonia_train_size+pneumonia_val_size]
    pneumonia_test = pneumonia_files[pneumonia_train_size+pneumonia_val_size:]
    
    
    normal_train_size = int(len(normal_files) * 0.8)
    normal_val_size = int(len(normal_files) * 0.1)
    normal_test_size = len(normal_files) - normal_train_size - normal_val_size
    
    normal_train = normal_files[:normal_train_size]
    normal_val = normal_files[normal_train_size:normal_train_size+normal_val_size]
    normal_test = normal_files[normal_train_size+normal_val_size:]
    
    print(f"\nDistribution originale divisée:")
    print(f"Train: {len(normal_train)} normales, {len(pneumonia_train)} pneumonies")
    print(f"Validation: {len(normal_val)} normales, {len(pneumonia_val)} pneumonies")
    print(f"Test: {len(normal_test)} normales, {len(pneumonia_test)} pneumonies")
    
    
    train_aug_needed = len(pneumonia_train) - len(normal_train)
    val_aug_needed = len(pneumonia_val) - len(normal_val)
    test_aug_needed = len(pneumonia_test) - len(normal_test)
    
    print(f"\nAugmentations nécessaires:")
    print(f"Train: {train_aug_needed} images normales supplémentaires")
    print(f"Validation: {val_aug_needed} images normales supplémentaires")
    print(f"Test: {test_aug_needed} images normales supplémentaires")
    
    
    def augment_dataset(source_images, dest_folder, count_needed):
        """Augmente un ensemble d'images pour atteindre le nombre souhaité"""
        nonlocal total_processed, successful, failed
        
        aug_count = 0
        aug_types = ['flip', 'rotate10', 'rotate-10', 'zoom', 'shift', 'brightness']
        
        print(f"Génération de {count_needed} images augmentées pour {dest_folder}...")
        
        
        aug_per_image = (count_needed // len(source_images)) + 1
        
        for src_path in source_images:
            if aug_count >= count_needed:
                break
                
            for aug_id in range(aug_per_image):
                if aug_count >= count_needed:
                    break
                    
                aug_type = aug_types[aug_id % len(aug_types)]
                filename = os.path.basename(src_path)
                name, ext = os.path.splitext(filename)
                aug_filename = f"{name}_aug{aug_id}{ext}"
                dst_path = os.path.join(dest_folder, 'NORMAL', aug_filename)
                
                try:
                    
                    img = Image.open(src_path)
                    
                    
                    if img.mode == 'RGB':
                        img = img.convert('L')
                    
                    
                    if aug_type == 'flip':
                        img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    elif aug_type == 'rotate10':
                        img = img.rotate(10, resample=Image.BICUBIC, expand=False)
                    elif aug_type == 'rotate-10':
                        img = img.rotate(-10, resample=Image.BICUBIC, expand=False)
                    elif aug_type == 'zoom':
                        width, height = img.size
                        crop_width = int(width * 0.9)
                        crop_height = int(height * 0.9)
                        left = (width - crop_width) // 2
                        top = (height - crop_height) // 2
                        img = img.crop((left, top, left + crop_width, top + crop_height))
                        img = img.resize((width, height), Image.LANCZOS)
                    elif aug_type == 'shift':
                        width, height = img.size
                        shift = int(width * 0.1)
                        img = img.crop((shift, 0, width, height))
                        img = img.resize((width, height), Image.LANCZOS)
                    elif aug_type == 'brightness':
                        img_array = np.array(img)
                        factor = 0.8 + random.random() * 0.4  
                        img_array = np.clip(img_array * factor, 0, 255)
                        img = Image.fromarray(img_array.astype(np.uint8))
                    
                    
                    img = img.resize(TARGET_SIZE, Image.LANCZOS)
                    
                    
                    img_array = np.array(img)
                    img_array = img_array / 255.0
                    normalized_img = Image.fromarray((img_array * 255).astype(np.uint8))
                    
                    
                    normalized_img.save(dst_path)
                    
                    aug_count += 1
                    total_processed += 1
                    successful += 1
                    
                    
                    if aug_count % 100 == 0:
                        print(f"  - {aug_count}/{count_needed} augmentations générées...")
                        
                except Exception as e:
                    print(f"Erreur lors de l'augmentation de {src_path}: {e}")
                    total_processed += 1
                    failed += 1
        
        return aug_count
    
    
    def process_original_images(src_images, dest_folder, class_name):
        nonlocal total_processed, successful, failed
        
        print(f"Traitement de {len(src_images)} images {class_name} pour {dest_folder}...")
        
        for i, src_path in enumerate(src_images):
            filename = os.path.basename(src_path)
            dst_path = os.path.join(dest_folder, class_name, filename)
            
            
            if i > 0 and i % 100 == 0:
                print(f"  - {i}/{len(src_images)} images traitées...")
            
            
            total_processed += 1
            if preprocess_image(src_path, dst_path):
                successful += 1
            else:
                failed += 1
    
    
    process_original_images(normal_train, preprocessed_train, 'NORMAL')
    process_original_images(pneumonia_train, preprocessed_train, 'PNEUMONIA')
    process_original_images(normal_val, preprocessed_val, 'NORMAL')
    process_original_images(pneumonia_val, preprocessed_val, 'PNEUMONIA')
    process_original_images(normal_test, preprocessed_test, 'NORMAL')
    process_original_images(pneumonia_test, preprocessed_test, 'PNEUMONIA')
    
    
    aug_train = augment_dataset(normal_train, preprocessed_train, train_aug_needed)
    aug_val = augment_dataset(normal_val, preprocessed_val, val_aug_needed)
    aug_test = augment_dataset(normal_test, preprocessed_test, test_aug_needed)
    
    
    elapsed_time = time.time() - start_time
    
    print("\nRésumé du prétraitement optimisé:")
    print(f"Temps total: {elapsed_time:.2f} secondes")
    print(f"Images traitées: {total_processed}")
    print(f"Succès: {successful}")
    print(f"Échecs: {failed}")
    print(f"Taux de réussite: {(successful/total_processed)*100:.2f}%")
    
    
    train_normal_count = len(os.listdir(os.path.join(preprocessed_train, 'NORMAL')))
    train_pneumonia_count = len(os.listdir(os.path.join(preprocessed_train, 'PNEUMONIA')))
    val_normal_count = len(os.listdir(os.path.join(preprocessed_val, 'NORMAL')))
    val_pneumonia_count = len(os.listdir(os.path.join(preprocessed_val, 'PNEUMONIA')))
    test_normal_count = len(os.listdir(os.path.join(preprocessed_test, 'NORMAL')))
    test_pneumonia_count = len(os.listdir(os.path.join(preprocessed_test, 'PNEUMONIA')))
    
    print("\nDistribution finale (toutes images originales conservées + équilibre 50/50):")
    print(f"Train: {train_normal_count} normales, {train_pneumonia_count} pneumonies")
    print(f"Validation: {val_normal_count} normales, {val_pneumonia_count} pneumonies")
    print(f"Test: {test_normal_count} normales, {test_pneumonia_count} pneumonies")
    print(f"Total: {train_normal_count + train_pneumonia_count + val_normal_count + val_pneumonia_count + test_normal_count + test_pneumonia_count} images")


create_optimized_balanced_dataset()

# Analyse statistique du dataset prétraité

Cette cellule analyse le dataset prétraité pour vérifier les résultats de nos transformations. Nous observons que toutes les images ont désormais une taille uniforme de 256×256 pixels et sont en niveaux de gris. Les classes sont parfaitement équilibrées (ratio 1:1) dans chaque ensemble grâce à l'augmentation des images normales. Les différences de contraste entre images normales (61.04) et pneumonies (55.19) ont été préservées, caractéristique qui pourrait aider notre modèle à distinguer les classes.

In [ ]:
import os
import numpy as np
import pandas as pd
import time
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter


preprocessed_path = 'preprocessed_chest_xray'
folders = ['train', 'test', 'val']

def analyze_image(img_path):
    """Analyse une image et extrait ses caractéristiques"""
    try:
        img = Image.open(img_path)
        img_width, img_height = img.size
        img_mode = img.mode
        img_array = np.array(img)
        
        
        if 'train' in img_path:
            dataset = 'Train'
        elif 'test' in img_path:
            dataset = 'Test'
        else:
            dataset = 'Validation'
        
        
        is_augmented = 'aug' in os.path.basename(img_path)
        
        return {
            'width': img_width,
            'height': img_height,
            'mode': img_mode,
            'mean_pixel': np.mean(img_array),
            'std_pixel': np.std(img_array),
            'class': 'Normal' if 'NORMAL' in img_path else 'Pneumonia',
            'dataset': dataset,
            'augmented': is_augmented
        }
    except Exception as e:
        print(f"Erreur lors de l'analyse de {img_path}: {e}")
        return None


all_files = []
for folder in folders:
    
    normal_path = os.path.join(preprocessed_path, folder, 'NORMAL')
    if os.path.exists(normal_path):
        normal_files = [os.path.join(normal_path, f) for f in os.listdir(normal_path) 
                    if f.endswith(('.jpeg', '.jpg', '.png'))]
        all_files.extend(normal_files)
    
    
    pneumonia_path = os.path.join(preprocessed_path, folder, 'PNEUMONIA')
    if os.path.exists(pneumonia_path):
        pneumonia_files = [os.path.join(pneumonia_path, f) for f in os.listdir(pneumonia_path) 
                      if f.endswith(('.jpeg', '.jpg', '.png'))]
        all_files.extend(pneumonia_files)

start_time = time.time()
print(f"Analyse de {len(all_files)} images dans le dataset prétraité...")


results = []
for i, img_path in enumerate(all_files):
    if i % 500 == 0 and i > 0: 
        print(f"Analysé {i}/{len(all_files)} images...")
    result = analyze_image(img_path)
    if result:
        results.append(result)

elapsed_time = time.time() - start_time
print(f"Analyse terminée en {elapsed_time:.2f} secondes")


df = pd.DataFrame(results)


normal_df = df[df['class'] == 'Normal']
pneumonia_df = df[df['class'] == 'Pneumonia']
original_normal_df = normal_df[~normal_df['augmented']]
augmented_normal_df = normal_df[normal_df['augmented']]


print(f"\n===== TAILLE DU DATASET =====")
print(f"Total images: {len(df)}")
print(f"Images normales: {len(normal_df)} (dont {len(original_normal_df)} originales, {len(augmented_normal_df)} augmentées)")
print(f"Images pneumonie: {len(pneumonia_df)}")


for dataset in ['Train', 'Test', 'Validation']:
    dataset_df = df[df['dataset'] == dataset]
    dataset_normal = dataset_df[dataset_df['class'] == 'Normal']
    dataset_pneumonia = dataset_df[dataset_df['class'] == 'Pneumonia']
    
    original_normal = dataset_normal[~dataset_normal['augmented']]
    augmented_normal = dataset_normal[dataset_normal['augmented']]
    
    print(f"\n===== ENSEMBLE {dataset.upper()} ({len(dataset_df)} images) =====")
    print(f"Normal: {len(dataset_normal)} images (dont {len(original_normal)} originales, {len(augmented_normal)} augmentées)")
    print(f"Pneumonie: {len(dataset_pneumonia)} images")
    ratio = len(dataset_normal)/len(dataset_pneumonia)
    print(f"Ratio Normal:Pneumonie = {ratio:.2f}:1")


print("\n===== DIMENSIONS =====")
print(f"Largeur moyenne (Normal originales): {original_normal_df['width'].mean():.1f}px")
print(f"Largeur moyenne (Normal augmentées): {augmented_normal_df['width'].mean():.1f}px")
print(f"Largeur moyenne (Pneumonie): {pneumonia_df['width'].mean():.1f}px")
print(f"Hauteur moyenne (Normal originales): {original_normal_df['height'].mean():.1f}px")
print(f"Hauteur moyenne (Normal augmentées): {augmented_normal_df['height'].mean():.1f}px")
print(f"Hauteur moyenne (Pneumonie): {pneumonia_df['height'].mean():.1f}px")


print("\n===== LUMINOSITÉ =====")
print(f"Luminosité moyenne (Normal originales): {original_normal_df['mean_pixel'].mean():.2f}")
print(f"Luminosité moyenne (Normal augmentées): {augmented_normal_df['mean_pixel'].mean():.2f}")
print(f"Luminosité moyenne (Pneumonie): {pneumonia_df['mean_pixel'].mean():.2f}")


print("\n===== CONTRASTE =====")
print(f"Contraste moyen (Normal originales): {original_normal_df['std_pixel'].mean():.2f}")
print(f"Contraste moyen (Normal augmentées): {augmented_normal_df['std_pixel'].mean():.2f}")
print(f"Contraste moyen (Pneumonie): {pneumonia_df['std_pixel'].mean():.2f}")


print("\n===== VARIABILITÉ ENTRE IMAGES =====")
print(f"Variabilité de luminosité (Normal originales): {original_normal_df['mean_pixel'].std():.2f}")
print(f"Variabilité de luminosité (Normal augmentées): {augmented_normal_df['mean_pixel'].std():.2f}")
print(f"Variabilité de luminosité (Pneumonie): {pneumonia_df['mean_pixel'].std():.2f}")
print(f"Variabilité de contraste (Normal originales): {original_normal_df['std_pixel'].std():.2f}")
print(f"Variabilité de contraste (Normal augmentées): {augmented_normal_df['std_pixel'].std():.2f}")
print(f"Variabilité de contraste (Pneumonie): {pneumonia_df['std_pixel'].std():.2f}")


print("\n===== MODES D'IMAGE =====")
print(df['mode'].value_counts())


all_sizes = df.groupby(['width', 'height']).size().reset_index(name='count')
print("\n===== VARIÉTÉ DE TAILLES =====")
print(f"Nombre de dimensions différentes: {len(all_sizes)}")
print(f"Dimensions les plus courantes: {all_sizes.sort_values('count', ascending=False).head(1).iloc[0]['width']}×{all_sizes.sort_values('count', ascending=False).head(1).iloc[0]['height']}")
print(f"Dimensions min: {df['width'].min()}×{df['height'].min()}")
print(f"Dimensions max: {df['width'].max()}×{df['height'].max()}")


plt.figure(figsize=(16, 7))


plt.subplot(1, 2, 1)
plt.hist(pneumonia_df['mean_pixel'], bins=30, alpha=0.7, color='red', label='Pneumonie')
plt.hist(original_normal_df['mean_pixel'], bins=30, alpha=0.7, color='blue', label='Normal (Original)')
plt.hist(augmented_normal_df['mean_pixel'], bins=30, alpha=0.7, color='green', label='Normal (Augmenté)')
plt.xlabel('Luminosité moyenne', fontsize=12)
plt.ylabel('Nombre d\'images', fontsize=12)
plt.title('Distribution des luminosités', fontsize=14)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)


plt.subplot(1, 2, 2)
plt.hist(pneumonia_df['std_pixel'], bins=30, alpha=0.7, color='red', label='Pneumonie')
plt.hist(original_normal_df['std_pixel'], bins=30, alpha=0.7, color='blue', label='Normal (Original)')
plt.hist(augmented_normal_df['std_pixel'], bins=30, alpha=0.7, color='green', label='Normal (Augmenté)')
plt.xlabel('Contraste (écart-type)', fontsize=12)
plt.ylabel('Nombre d\'images', fontsize=12)
plt.title('Distribution des contrastes', fontsize=14)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)


plt.tight_layout()
plt.show()


print("\n===== CONCLUSION DE L'ANALYSE =====")
print("1. Équilibre des classes: Parfaitement équilibré dans chaque ensemble (ratio 1:1)")
print("2. Dimensions: Toutes les images ont été standardisées à la même taille")
print("3. Distribution par ensemble: Respecte la répartition 80/10/10 (train/val/test)")
print("4. Images augmentées: Comportement similaire aux originales en termes de contraste et luminosité")